#### 读取RT Dicom文件的头文件信息

In [ ]:
import os
import pydicom as dicom

file_path = r'../RD.1.2.246.352.71.7.849455740085.3750465.20180928164145.dcm'
dcm = dicom.read_file(file_path)
print(dcm.SOPInstanceUID)
print(dcm.pixel_array.shape)

#### RT Plan头文件显示

In [42]:
import pydicom
import sys
from PySide2 import QtGui, QtWidgets
import collections


class DicomTree(object):
    def __init__(self, filename):
        self.filename = filename

    def show_tree(self):
        ds = self.dicom_to_dataset(self.filename)
        dic = self.dataset_to_dic(ds)
        model = self.dic_to_model(dic)
        self.display(model)

    def array_to_model(self, array):
        model = QtGui.QStandardItemModel()
        parentItem = model.invisibleRootItem()
        for ntuple in array:
            tag = ntuple[0]
            value = ntuple[1]
            if isinstance(value, dict):
                self.recurse_dic_to_item(value, parentItem)
            else:
                item = QtGui.QStandardItem(tag + str(value))
                parentItem.appendRow(item)
        return parentItem

    def dic_to_model(self, dic):
        model = QtGui.QStandardItemModel()
        parentItem = model.invisibleRootItem()
        self.recurse_dic_to_item(dic, parentItem)
        return model

    def dataset_to_array(self, dataset):
        array = []
        for data_element in dataset:
            array.append(self.data_element_to_dic(data_element))
        return array

    def recurse_dic_to_item(self, dic, parent):
        for k in dic:
            v = dic[k]
            if isinstance(v, dict):
                item = QtGui.QStandardItem(k + ':' + str(v))
                parent.appendRow(self.recurse_dic_to_item(v, item))
            else:
                item = QtGui.QStandardItem(k + ': ' + str(v))
                parent.appendRow(item)
        return parent

    def dicom_to_dataset(self, filename):
        dataset = pydicom.dcmread(filename, force=True)
        return dataset

    def data_element_to_dic(self, data_element):
        dic = collections.OrderedDict()
        if data_element.VR == "SQ":
            items = collections.OrderedDict()
            dic[data_element.name] = items
            i = 0
            for dataset_item in data_element:
                items['item ' + str(i)] = self.dataset_to_dic(dataset_item)
                i += 1
        elif data_element.name != 'Pixel Data':
            dic[data_element.name] = data_element.value
        return dic

    def dataset_to_dic(self, dataset):
        dic = collections.OrderedDict()
        for data_element in dataset:
            dic.update(self.data_element_to_dic(data_element))
        return dic

    def display(self, model):
        app = QtWidgets.QApplication.instance()

        # create QApplication if it doesnt exist
        if not app:
            app = QtWidgets.QApplication(sys.argv)
        tree = QtWidgets.QTreeView()
        tree.setModel(model)
        tree.show()
        app.exec_()
        return tree


def main():
    filename = r'../RP.1.2.246.352.71.5.849455740085.2415038.20180928163843.dcm'
    dicomTree = DicomTree(filename)
    dicomTree.show_tree()


if __name__ == "__main__":
    main()

#### 读取Dicom序列的几种方法

In [45]:
# 方法一：SimpleITK

import SimpleITK as sitk

def readdcm(filepath):
    reader = sitk.ImageSeriesReader()  # 读取数据端口
    reader.MetaDataDictionaryArrayUpdateOn()  # 这一步是加载公开的元信息
    reader.LoadPrivateTagsOn()  # 这一步是加载私有的元信息
    series_id = reader.GetGDCMSeriesIDs(filepath)
    series_file_names = reader.GetGDCMSeriesFileNames(filepath, series_id[0])

    reader.SetFileNames(series_file_names)  # 读取名称
    images = reader.Execute()  # 读取数据
    # sitk.WriteImage(images, "T2_1.nii.gz")#保存为nii
    return images

readdcm(filepath)

In [53]:
# 方法二：dicom2nifti

import dicom2nifti     #直接转换，不考虑序列调转的情况
import dicom2nifti.common as common

savepath = r"../00010/"  #保存路径   
inputpath = r"../00010/CT"

dicom_input = common.read_dicom_directory(inputpath)  # 读取整个dicon文件夹中每个dicom的头文件信息

dicom2nifti.dicom_series_to_nifti(inputpath, savepath, reorient_nifti=False)  # 可以控制床位方向


{'NII_FILE': 'C:\\Users\\Joker_ZXR\\Desktop\\data\\00010/',
 'NII': <nibabel.nifti1.Nifti1Image at 0x2744614ba30>,
 'MAX_SLICE_INCREMENT': 3.0}